In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader,random_split,Dataset
import torch.optim as optim
from tqdm import tqdm
from wideresnet import *


In [2]:
trainset = datasets.CIFAR10(root='/home/aminul/data1/', train=True, download=False, transform=transforms.ToTensor())
testset = datasets.CIFAR10(root='/home/aminul/data1/', train=False, download=False, transform=transforms.ToTensor())

labels_list = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
non_animal = [0,1,8,9]
device = 'cuda:0'

In [3]:
class NewDataset(Dataset):
    
    def __init__(self,data,transform=None):
        self.data = data
        
    def __len__(self):
        return len(self.data)    
    
    def __getitem__(self,idx):
        image = self.data[idx][0]
        label1 = self.data[idx][1]
        label2 = 0 if self.data[idx][1] in non_animal else 1
        label3 = 0 if self.data[idx][1] > 5 else 1
        return image, label1, label2, label3

In [4]:
new_trainset = NewDataset(trainset,non_animal)
new_testset = NewDataset(testset,non_animal)

train_loader = DataLoader(new_trainset, batch_size=100, shuffle=True)
test_loader = DataLoader(new_testset, batch_size=100, shuffle=True)

In [6]:
def train(net,trainloader,optim,criterion,epoch,device):
    net.train()
    train_loss,total,total_correct1,total_correct2,total_correct3 = 0,0,0,0,0
    
    for i,(inputs,tg1,tg2,tg3) in enumerate(tqdm(trainloader)):
        
        inputs,tg1,tg2,tg3 = inputs.to(device), tg1.to(device), tg2.to(device), tg3.to(device)
        optim.zero_grad()
        
        op1,op2,op3 = net(inputs)
        loss1 = criterion(op1,tg1)
        loss2 = criterion(op2,tg2)
        loss3 = criterion(op3,tg3)
        
        loss1.backward(retain_graph=True)
        loss2.backward(retain_graph=True)
        loss3.backward()
        
        optim.step()
        
        train_loss += loss1.item() + loss2.item() + loss3.item()
        _,pd1 = torch.max(op1.data,1)
        _,pd2 = torch.max(op2.data,1)
        _,pd3 = torch.max(op3.data,1)
        total_correct1 += (pd1 == tg1).sum().item()
        total_correct2 += (pd2 == tg2).sum().item()
        total_correct3 += (pd3 == tg3).sum().item()
        total += tg1.size(0)
    
    print("Epoch: [{}]  loss: [{:.2f}] Orig_Acc [{:.2f}] animal_Acc [{:.2f}] random_Acc [{:.2f}] ".format
                                                                          (epoch+1,train_loss/(i+1),
                                                                           (total_correct1*100/total),
                                                                          (total_correct2*100/total)
                                                                         ,(total_correct3*100/total)))
    return train_loss/(i+1)

In [7]:
def tester(net,testloader,optim,criterion,epoch,device):
    net.eval()
    test_loss,total,total_correct1,total_correct2,total_correct3 = 0,0,0,0,0
    
    for i,(inputs,tg1,tg2,tg3) in enumerate(tqdm(testloader)):
        
        inputs,tg1,tg2,tg3 = inputs.to(device), tg1.to(device), tg2.to(device), tg3.to(device)
        optim.zero_grad()
        
        op1,op2,op3 = net(inputs)
        loss1 = criterion(op1,tg1)
        loss2 = criterion(op2,tg2)
        loss3 = criterion(op3,tg3)
        
        test_loss += loss1.item() + loss2.item() + loss3.item()
        _,pd1 = torch.max(op1.data,1)
        _,pd2 = torch.max(op2.data,1)
        _,pd3 = torch.max(op3.data,1)
        total_correct1 += (pd1 == tg1).sum().item()
        total_correct2 += (pd2 == tg2).sum().item()
        total_correct3 += (pd3 == tg3).sum().item()
        total += tg1.size(0)
        
    acc1 = 100. * total_correct1 / total
    acc2 = 100. * total_correct2 / total
    acc3 = 100. * total_correct3 / total
    print("\nTest Epoch #%d Loss: %.4f Orig_Acc: %.2f%% animal_Acc: %.2f%% random_Acc: %.2f%%" %(epoch+1,
                                                                                                  test_loss/(i+1),
                                                                                                  acc1,acc2,acc3))
        
    return test_loss/(i+1), acc1, acc2, acc3

In [8]:
total_classes = [10,2,2]
net = WideResNet(depth=28,num_classes = total_classes,widen_factor=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001,momentum=0.9, weight_decay=5e-4)

In [9]:
num_epochs = 20
train_loss,test_loss = [],[]

for epoch in range(num_epochs):
    
    a = train(net,train_loader,optimizer,criterion,epoch,device)
    c,_,_,_ = tester(net,test_loader,optimizer,criterion,epoch,device)    
    
    train_loss.append(a), test_loss.append(c)

  1%|          | 1/100 [00:00<00:15,  6.28it/s]

Epoch: [1]  loss: [2.71] Orig_Acc [34.07] animal_Acc [85.98] random_Acc [65.96] 


  0%|          | 0/500 [00:00<?, ?it/s]


Test Epoch #1 Loss: 2.3915 Orig_Acc: 41.36% animal_Acc: 90.61% random_Acc: 69.56%


  1%|          | 1/100 [00:00<00:12,  7.90it/s]

...


Test Epoch #19 Loss: 1.5017 Orig_Acc: 77.97% animal_Acc: 96.42% random_Acc: 86.56%


  1%|          | 1/100 [00:00<00:12,  7.89it/s]

Epoch: [20]  loss: [0.02] Orig_Acc [99.72] animal_Acc [99.95] random_Acc [99.91] 


100%|██████████| 100/100 [00:13<00:00,  7.65it/s]


Test Epoch #20 Loss: 1.4501 Orig_Acc: 78.12% animal_Acc: 96.52% random_Acc: 88.54%
